In [75]:
import random
import torch
from utils.load_results import *
from utils.qualitative_utils import *
import pandas as pd
from matplotlib import pyplot as plt
plt.style.use('default')
from utils.analysis_from_interaction import *
import os
if not os.path.exists('analysis'):
    os.makedirs('analysis')

### Configurations

In [1]:
datasets = ['(3,4)', '(3,8)', '(3,16)', '(4,4)', '(4,8)', '(5,4)']
n_values = [4, 8, 16, 4, 8, 4]
n_attributes = [3, 3, 3, 4, 4, 5]
vocab_sizes = [16, 28, 52, 16, 28, 16]
n_epochs = 300
n_runs = 5
n_datasets = len(datasets)
paths = ['results/' + d + '_game_size_10_vsf_3' for d in datasets]

In [77]:
context_unaware = False # whether original or context_unaware simulations are evaluated
zero_shot = True # whether zero-shot simulations are evaluated
zero_shot_test = 'generic' # 'generic' or 'specific'
test_interactions = True # whether scores should be calculated on test interactions (only with zero shot)
zero_shot_test_ds = 'test_sampled_unscaled' # 'test' or 'test_sampled_unscaled' or 'test_unscaled' or 'test_fine' 
setting = ""
if context_unaware:
    setting = setting + 'context_unaware'
else:
    setting = setting + 'standard'
if zero_shot:
    setting = setting + '/zero_shot/' + zero_shot_test

### Determine vocab size and message reuse

In [13]:
# go through all datasets
for i, d in enumerate(datasets):
    print(d)
    for run in range(n_runs):
        path_to_run = paths[i] + '/' + str(setting) +'/' + str(run) + '/'
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
        path_to_interaction_test = (path_to_run + 'interactions/' + str(zero_shot_test_ds) +'/epoch_0/interaction_gpu0')
        interaction_train = torch.load(path_to_interaction_train)
        interaction_val = torch.load(path_to_interaction_val)
        interaction_test = torch.load(path_to_interaction_test)
        
        concepts_train = objects_to_concepts(interaction_train.sender_input, n_values=n_values[i])
        concepts_val = objects_to_concepts(interaction_val.sender_input, n_values=n_values[i])
        concepts_test = objects_to_concepts(interaction_test.sender_input, n_values=n_values[i])
        
        messages_train = retrieve_messages(interaction_train)
        messages_val = retrieve_messages(interaction_val)
        messages_test = retrieve_messages(interaction_test)
    
        symbol_counts_train = count_symbols(messages_train)
        symbol_counts_val = count_symbols(messages_val)
        symbol_counts_test = count_symbols(messages_test)
        symbol_counts = [symbol_counts_train, symbol_counts_val, symbol_counts_test]
        pickle.dump(symbol_counts, open(path_to_run + 'symbol_counts_' + str(zero_shot_test_ds) + '.pkl', 'wb'))
        actual_vocab_size = len(symbol_counts_train + symbol_counts_val + symbol_counts_test)
        print(actual_vocab_size, "symbols from the alphabet have been actually used during training, validation and testing.")
        pickle.dump(actual_vocab_size, open(path_to_run + 'vocab_size_' + str(zero_shot_test_ds) + '.pkl', 'wb'))
        
        # consider train and validation messages together
        messages_train_val = messages_train +  messages_val
        # consider only unique messages
        messages_train_val_unique = get_unique_message_set(messages_train_val)
        #print("messages train val", len(messages_train_val), len(messages_train_val_unique))
        messages_test_unique = get_unique_message_set(messages_test)
        #print("messages test", len(messages_test), len(messages_test_unique))
        # total messages
        messages_total = messages_train_val +  messages_test
        messages_total_unique = get_unique_message_set(messages_total)
        
        # concepts
        concepts_train_unique = get_unique_concept_set(concepts_train)
        concepts_val_unique = get_unique_concept_set(concepts_val)
        concepts_test_unique = get_unique_concept_set(concepts_test)
        #print("concepts", len(concepts_test), len(concepts_test_unique))
        concepts_total = concepts_train + concepts_val + concepts_test
        concepts_total_unique = get_unique_concept_set(concepts_total)
        num_of_concepts = [len(concepts_train_unique), len(concepts_val_unique), len(concepts_test_unique), len(concepts_total_unique), len(concepts_total)]
        pickle.dump(num_of_concepts, open(path_to_run + 'num_of_concepts_' + str(zero_shot_test_ds) + '.pkl', 'wb'))
        
        # messages reused in testing:
        intersection = messages_train_val_unique & messages_test_unique
        
        # messages only used in training:
        difference_train = messages_train_val_unique - messages_test_unique
        
        # messages only used in testing:
        difference_test = messages_test_unique - messages_train_val_unique
        print(len(difference_test), "novel messages used for the", len(concepts_test_unique), "novel concepts")
        
        message_reuse = [len(intersection), len(difference_train), len(difference_test), len(concepts_test_unique), (len(difference_test)/len(concepts_test_unique)), len(messages_test_unique)]
        pickle.dump(message_reuse, open(path_to_run + 'message_reuse_' + str(zero_shot_test_ds) + '.pkl', 'wb'))

(3,4)
16 symbols from the alphabet have been actually used during training, validation and testing.
31 novel messages used for the 64 novel concepts
15 symbols from the alphabet have been actually used during training, validation and testing.
19 novel messages used for the 64 novel concepts
14 symbols from the alphabet have been actually used during training, validation and testing.
19 novel messages used for the 64 novel concepts
15 symbols from the alphabet have been actually used during training, validation and testing.
28 novel messages used for the 64 novel concepts
16 symbols from the alphabet have been actually used during training, validation and testing.
28 novel messages used for the 64 novel concepts
(3,8)
27 symbols from the alphabet have been actually used during training, validation and testing.
218 novel messages used for the 512 novel concepts
25 symbols from the alphabet have been actually used during training, validation and testing.
180 novel messages used for the 51

In [139]:
message_reuse_dict = {'intersection': [], 'difference train': [], 'difference test': [], 'concepts test unique': [], 'test ratio': [], 'messages test unique': [],
                      'reuse rate': [], 'novelty rate': [], 'total ratio': []}
for i, d in enumerate(datasets):
    intersection, train_difference, test_difference, test_concepts, test_ratio, test_messages, reuse_rate, novelty_rate, total_ratio = [], [], [], [], [], [], [], [], []
    for run in range(n_runs):
        path_to_run = paths[i] + '/' + str(setting) +'/' + str(run) + '/'
        message_reuse = pickle.load(open(path_to_run + 'message_reuse_' + str(zero_shot_test_ds) + '.pkl', 'rb'))
        intersection.append(message_reuse[0]) # messages reused in testing
        train_difference.append(message_reuse[1]) # messages only used in training
        test_difference.append(message_reuse[2]) # messages only used in testing
        test_concepts.append(message_reuse[3]) # total number of concepts used in testing
        test_ratio.append(message_reuse[4]) 
        test_messages.append(message_reuse[5]) # total number of messages used in testing
        reuse_rate.append(message_reuse[0]/message_reuse[5])
        novelty_rate.append(message_reuse[2]/message_reuse[5])
        total_ratio.append(message_reuse[5]/message_reuse[3]) # test_messages / test_concepts (novel unique messages & concepts)
    message_reuse_dict['intersection'].append(intersection)
    message_reuse_dict['difference train'].append(train_difference)
    message_reuse_dict['difference test'].append(test_difference)
    message_reuse_dict['concepts test unique'].append(test_concepts)
    message_reuse_dict['test ratio'].append(test_ratio)
    message_reuse_dict['messages test unique'].append(test_messages)
    message_reuse_dict['reuse rate'].append(reuse_rate)
    message_reuse_dict['novelty rate'].append(novelty_rate)
    message_reuse_dict['total ratio'].append(total_ratio)

In [140]:
message_reuse = [message_reuse_dict['concepts test unique'], message_reuse_dict['messages test unique'], message_reuse_dict['total ratio'], message_reuse_dict['reuse rate'], message_reuse_dict['novelty rate']]

# Convert the list to a NumPy array
mess_reuse_array = np.array(message_reuse)

# Compute means and standard deviations over the five runs
means = np.mean(mess_reuse_array, axis=-1)
std_devs = np.std(mess_reuse_array, axis=-1)

# Row names and column names
row_names = ["D(3,4)", "D(3,8)", "D(3,16)", "D(4,4)", "D(4,8)", "D(5,4)"]
col_names = ["test concepts", "test messages", "message-concept ratio", "reuse rate","novelty rate"]

# Prepare the data for the DataFrames
data = []

# iterate over datasets
for i in range(means.shape[1]):
    row = []
    # iterate over conditions
    for j in range(means.shape[0]):
        if j > 1:
            formatted_value = f"{means[j, i]:.2f} $\\pm$ {std_devs[j, i]:.2f}"
        elif j == 0:
            formatted_value = f"{int(means[j, i])}"
        else:
            formatted_value = f"{means[j, i]:.1f} $\\pm$ {std_devs[j, i]:.1f}"
        row.append(formatted_value)
    data.append(row)

# Create DataFrames
df = pd.DataFrame(data, index=row_names, columns=col_names)

# Convert DataFrames to LaTeX tables
latex_table = df.to_latex(index=True, escape=False)

print(latex_table)

\begin{tabular}{llllll}
\toprule
{} & test concepts &   test messages & message-concept ratio &       reuse rate &     novelty rate \\
\midrule
D(3,4)  &            12 &  11.8 $\pm$ 0.4 &       0.98 $\pm$ 0.03 &  0.80 $\pm$ 0.19 &  0.20 $\pm$ 0.19 \\
D(3,8)  &            24 &  22.2 $\pm$ 0.4 &       0.93 $\pm$ 0.02 &  0.83 $\pm$ 0.07 &  0.17 $\pm$ 0.07 \\
D(3,16) &            48 &  44.4 $\pm$ 0.8 &       0.93 $\pm$ 0.02 &  0.75 $\pm$ 0.06 &  0.25 $\pm$ 0.06 \\
D(4,4)  &            16 &  15.6 $\pm$ 0.8 &       0.97 $\pm$ 0.05 &  0.74 $\pm$ 0.14 &  0.26 $\pm$ 0.14 \\
D(4,8)  &            32 &  28.8 $\pm$ 3.1 &       0.90 $\pm$ 0.10 &  0.93 $\pm$ 0.06 &  0.07 $\pm$ 0.06 \\
D(5,4)  &            20 &  19.6 $\pm$ 0.5 &       0.98 $\pm$ 0.02 &  1.00 $\pm$ 0.00 &  0.00 $\pm$ 0.00 \\
\bottomrule
\end{tabular}


/var/folders/k6/03rlh8jd6nqbws0_xg9jh20w0000gq/T/ipykernel_9677/4058811893.py:35: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  latex_table = df.to_latex(index=True, escape=False)


### Symbol reuse
All symbols are (re)used, i.e. they all encode relevant information. This is why a qualitative analysis of messages makes more sense.

In [80]:
# go through all datasets
for i, d in enumerate(datasets):
    print(d)
    # get random qualitative samples
    # to specific: all indices should be fixed
    if zero_shot_test == 'specific':
        n_fixed = n_attributes[i]
        fixed_indices = list(range(0, n_attributes[i])) # all attributes fixed
        fixed_values = random.choices(range(0, n_values[i]), k=n_fixed) # define fixed values for these indices
    # to generic: only one index fixed, which one is randomly determined
    elif zero_shot_test == 'generic':
        n_fixed = 1
        fixed_indices = random.sample(range(0, n_attributes[i]), k=n_fixed) # select which attribute is fixed
        fixed_values = random.choices(range(0, n_values[i]), k=n_fixed) # define a fixed value for this index
    #print(fixed_indices, fixed_values)
    for run in range(n_runs):
        path_to_run = paths[i] + '/' + str(setting) +'/' + str(run) + '/'
        path_to_interaction_train = (path_to_run + 'interactions/train/epoch_' + str(n_epochs) + '/interaction_gpu0')
        path_to_interaction_val = (path_to_run + 'interactions/validation/epoch_' + str(n_epochs) + '/interaction_gpu0')
        path_to_interaction_test = (path_to_run + 'interactions/' + str(zero_shot_test_ds) + '/epoch_0/interaction_gpu0')
        interaction_train = torch.load(path_to_interaction_train)
        interaction_val = torch.load(path_to_interaction_val)
        interaction_test = torch.load(path_to_interaction_test)
        
        # retrieve mapping between attribute-value pairs and symbols
        favorite_symbol, mutual_information = symbol_frequency_MI(interaction_train, n_attributes=n_attributes[i], n_values=n_values[i], vocab_size=vocab_sizes[i])
        print(favorite_symbol, mutual_information)
        
        messages = interaction_test.message.argmax(dim=-1)
        messages = [msg.tolist() for msg in messages]
        sender_input = interaction_test.sender_input
        n_targets = int(sender_input.shape[1]/2)
        # get target objects and fixed vectors to re-construct concepts
        target_objects = sender_input[:, :n_targets]
        target_objects = k_hot_to_attributes(target_objects, n_values[i])
        # concepts are defined by a list of target objects (here one sampled target object) and a fixed vector
        (objects, fixed) = retrieve_concepts_sampling(target_objects, all_targets=True)
        concepts = list(zip(objects, fixed))
        
        # get distractor objects to re-construct context conditions
        distractor_objects = sender_input[:, n_targets:]
        distractor_objects = k_hot_to_attributes(distractor_objects, n_values[i])
        context_conds = retrieve_context_condition(objects, fixed, distractor_objects)
        
        all_for_this_concept = []
        for idx, (t_objects, t_fixed) in enumerate(concepts):
            if sum(t_fixed) == n_fixed and all(t_fixed[fixed_index] == 1 for fixed_index in fixed_indices):
                for t_object in t_objects:
                    if all(t_object[fixed_index] == fixed_values[j] for j, fixed_index in enumerate(fixed_indices)):
                        all_for_this_concept.append((idx, t_object, t_fixed, context_conds[idx], messages[idx]))
                        fixed = t_fixed
        if len(all_for_this_concept) > 0:
            #sample = random.sample(all_for_this_concept, 20)
            sample = all_for_this_concept
            column_names = ['game_nr', 'object', 'fixed indices', 'context condition', 'message']
            sample_df = pd.DataFrame(sample, columns=column_names)
            # find out which messages have been used how often (once in test dataset test_sampled_unscaled)
            message_counts = sample_df.message.apply(tuple).value_counts()/10 # divide by game size because above single objects are taken, but we are interested in concepts (i.e. sets of game_size=10 objects)
            messages = message_counts.index.tolist()
            counts = message_counts.values.tolist()
            cond_indices = np.arange(0, len(messages)*10, 10)
            context_conds = sample_df['context condition'][cond_indices]
            used_symbols = look_up_values(fixed_indices, fixed_values, favorite_symbol)
            symbol_MI = look_up_values(fixed_indices, fixed_values, mutual_information)
            df_concept = pd.DataFrame({
                'fixed indices': [fixed_indices], 
                'fixed values': [fixed_values]
            })
            df_messages = pd.DataFrame({
                'context condition': context_conds.values.tolist(),
                'message': messages, 
                'counts': counts
            })
            df_symbols = pd.DataFrame({
                'symbols': used_symbols, 
                'symbol MI': symbol_MI
            })
            df = pd.concat([df_concept, df_messages, df_symbols])
            print(df.to_latex(float_format=lambda x: '{:,.0f}'.format(x) if x % 1 == 0 else '{:,.4f}'.format(x)))
            # df.to_csv('analysis/' + str(zero_shot_test) + '/quali_' + str(d) + '_' + str(zero_shot_test) + '_' + str(run) + '_' + str(fixed_indices) + ',' + str(fixed_values) + 'message_symbol_counts.csv', index=False)
            # print('saved ' + 'analysis/' + str(zero_shot_test) + '/quali_' + str(d) + '_' + str(zero_shot_test) + '_' + str(run) + '_' + str(fixed_indices) + ',' + str(fixed_values) + 'message_symbol_counts.csv')
        else:
            raise ValueError("sample for dataset " + str(d) + " could not be generated")

(3,4)
{'00': 10, '01': 14, '02': 10, '03': 14, '10': 12, '11': 2, '12': 12, '13': 4, '20': 1, '21': 3, '22': 8, '23': 6} {'00': 0.4864617121149391, '01': 0.31809888818804016, '02': 0.07395326041506703, '03': 0.14001071181742006, '10': 0.4040838988255297, '11': 0.19390815354861027, '12': 0.09202954517331902, '13': 0.10640461426172829, '20': 0.43360799313014875, '21': 0.48540994902736895, '22': 0.843152773807735, '23': 0.4893656672024036}
\begin{tabular}{lllrlrrr}
\toprule
{} & fixed indices & fixed values &  context condition &         message &  counts &  symbols &  symbol MI \\
\midrule
0 &           [0] &          [0] &                NaN &             NaN &     NaN &      NaN &        NaN \\
0 &           NaN &          NaN &                  0 &  (10, 10, 4, 0) &       1 &      NaN &        NaN \\
0 &           NaN &          NaN &                NaN &             NaN &     NaN &       10 &     0.4865 \\
\bottomrule
\end{tabular}


/var/folders/k6/03rlh8jd6nqbws0_xg9jh20w0000gq/T/ipykernel_57624/3501616567.py:79: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex(float_format=lambda x: '{:,.0f}'.format(x) if x % 1 == 0 else '{:,.4f}'.format(x)))


{'00': 12, '01': 10, '02': 2, '03': 2, '10': 8, '11': 7, '12': 15, '13': 6, '20': 1, '21': 1, '22': 1, '23': 1} {'00': 0.5076952469136099, '01': 0.5339662976146126, '02': 0.13113573623483843, '03': 0.1589164480896561, '10': 0.4126539159502428, '11': 0.3257097467158606, '12': 0.4316118772835139, '13': 0.5407662034395362, '20': 0.12089150295451703, '21': 0.1105564514853039, '22': 0.07568368535857187, '23': 0.10874364975904957}
\begin{tabular}{lllrlrrr}
\toprule
{} & fixed indices & fixed values &  context condition &         message &  counts &  symbols &  symbol MI \\
\midrule
0 &           [0] &          [0] &                NaN &             NaN &     NaN &      NaN &        NaN \\
0 &           NaN &          NaN &                  0 &  (12, 2, 11, 0) &       1 &      NaN &        NaN \\
0 &           NaN &          NaN &                NaN &             NaN &     NaN &       12 &     0.5077 \\
\bottomrule
\end{tabular}

{'00': 5, '01': 13, '02': 1, '03': 11, '10': 12, '11': 9, '12':

/var/folders/k6/03rlh8jd6nqbws0_xg9jh20w0000gq/T/ipykernel_57624/3501616567.py:79: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex(float_format=lambda x: '{:,.0f}'.format(x) if x % 1 == 0 else '{:,.4f}'.format(x)))
/var/folders/k6/03rlh8jd6nqbws0_xg9jh20w0000gq/T/ipykernel_57624/3501616567.py:79: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex(float_format=lambda x: '{:,.0f}'.format(x) if x % 1 == 0 else '{:,.4f}'.format(x)))


{'00': 4, '01': 3, '02': 4, '03': 4, '10': 9, '11': 15, '12': 12, '13': 2, '20': 6, '21': 5, '22': 5, '23': 5} {'00': 0.16392180669118572, '01': 0.21847864503195494, '02': 0.15824209638204745, '03': 0.6200973094555102, '10': 0.2984878967694426, '11': 0.9031759276273484, '12': 0.4081297610350643, '13': 0.2678195391297006, '20': 0.02245599461959702, '21': 0.012759122475323853, '22': 0.02170244773624464, '23': 0.012216510445177032}
\begin{tabular}{lllrlrrr}
\toprule
{} & fixed indices & fixed values &  context condition &       message &  counts &  symbols &  symbol MI \\
\midrule
0 &           [0] &          [0] &                NaN &           NaN &     NaN &      NaN &        NaN \\
0 &           NaN &          NaN &                  0 &  (8, 8, 8, 0) &       1 &      NaN &        NaN \\
0 &           NaN &          NaN &                NaN &           NaN &     NaN &        4 &     0.1639 \\
\bottomrule
\end{tabular}

{'00': 10, '01': 14, '02': 9, '03': 10, '10': 8, '11': 9, '12': 11,

...

/var/folders/k6/03rlh8jd6nqbws0_xg9jh20w0000gq/T/ipykernel_57624/3501616567.py:79: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex(float_format=lambda x: '{:,.0f}'.format(x) if x % 1 == 0 else '{:,.4f}'.format(x)))


{'00': 6, '01': 2, '02': 5, '03': 2, '10': 5, '11': 6, '12': 6, '13': 2, '20': 12, '21': 13, '22': 8, '23': 2, '30': 11, '31': 4, '32': 14, '33': 10, '40': 1, '41': 7, '42': 15, '43': 9} {'00': 0.1675766770288002, '01': 0.18860270650679506, '02': 0.27588562735858146, '03': 0.03711106047272884, '10': 0.26238270802975583, '11': 0.037729968889987524, '12': 0.33733302332954596, '13': 0.22276852025113522, '20': 0.8212828180353039, '21': 0.9764627697790705, '22': 0.12231937848697876, '23': 0.15556645003565592, '30': 0.6837643722676454, '31': 0.7298767996749504, '32': 0.6637157309411021, '33': 0.7288473357752928, '40': 0.9232490391007947, '41': 0.8747831395098009, '42': 0.8787003004522649, '43': 0.8325487857048027}
\begin{tabular}{lllrlrrr}
\toprule
{} & fixed indices & fixed values &  context condition &             message &  counts &  symbols &  symbol MI \\
\midrule
0 &           [2] &          [2] &                NaN &                 NaN &     NaN &      NaN &        NaN \\
0 &        

/var/folders/k6/03rlh8jd6nqbws0_xg9jh20w0000gq/T/ipykernel_57624/3501616567.py:79: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex(float_format=lambda x: '{:,.0f}'.format(x) if x % 1 == 0 else '{:,.4f}'.format(x)))


{'00': 8, '01': 8, '02': 15, '03': 7, '10': 9, '11': 4, '12': 2, '13': 9, '20': 14, '21': 10, '22': 10, '23': 14, '30': 3, '31': 1, '32': 1, '33': 5, '40': 4, '41': 15, '42': 9, '43': 2} {'00': 0.10954319170400681, '01': 0.9755935610264024, '02': 0.37146637461462934, '03': 0.6535010186904553, '10': 0.27154712748642607, '11': 0.2786939064607155, '12': 0.38625243312836827, '13': 0.22661170701131242, '20': 0.5184383336488577, '21': 0.44041171046794825, '22': 0.09214921746842394, '23': 0.07008474854274895, '30': 0.2930429884953001, '31': 0.3085023046328236, '32': 0.21055125799205945, '33': 0.33904294884013486, '40': 0.13338221518421212, '41': 0.26978449448670044, '42': 0.09655621506330056, '43': 0.10843263619163855}
\begin{tabular}{lllrlrrr}
\toprule
{} & fixed indices & fixed values &  context condition &                 message &  counts &  symbols &  symbol MI \\
\midrule
0 &           [2] &          [2] &                NaN &                     NaN &     NaN &      NaN &        NaN \\

/var/folders/k6/03rlh8jd6nqbws0_xg9jh20w0000gq/T/ipykernel_57624/3501616567.py:79: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  print(df.to_latex(float_format=lambda x: '{:,.0f}'.format(x) if x % 1 == 0 else '{:,.4f}'.format(x)))
